In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import random
import collections

def load_dataset(dataset, target_dtype, features_dtype, target_column=-1):
    Dataset = collections.namedtuple('Dataset', ['data', 'target'])
    data, target = [], []
    for row in dataset:
        target.append(row.pop(target_column))
        data.append(np.asarray(row, dtype=features_dtype))
    
    target = np.array(target, dtype=target_dtype)
    data = np.array(data)
    return Dataset(data=data, target=target)

# Data sets
#IRIS_TRAINING = "time-series.csv"
#IRIS_TEST = "time-series.csv"

file=open("time-series.csv","r")
data=list()
for line in file:
    data.append(line.split(','))
file.close()
random.shuffle(data)
train_data = data[:int((len(data)+1)*.80)] #Remaining 80% to training set
test_data = data[int(len(data)*.80+1):] #Splits 20% data to test set

# Load datasets.
training_set = load_dataset(
  dataset=train_data,
  target_dtype=np.float32,
  features_dtype=np.float32)
test_set = load_dataset(
  dataset=test_data,
  target_dtype=np.float32,
  features_dtype=np.float32)

In [2]:
# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=24)]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[30, 50, 20, 10],
                                          n_classes=2,
                                          model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001CB16D03C18>, '_task_type': None, '_environment': 'local', '_task_id': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_master': ''}


In [3]:
# Define the training inputs
def get_train_inputs():
    training_set.target[training_set.target < 5] = 0
    training_set.target[training_set.target > 5] = 1
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    return x, y

In [4]:
# Fit model.
classifier.fit(input_fn=get_train_inputs, steps=2000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:step = 1, loss = 5.05058e+06
INFO:tensorflow:global_step/sec: 71.264
INFO:tensorflow:step = 101, loss = 8578.08
INFO:tensorflow:global_step/sec: 71.9063
INFO:tensorflow:step = 201, loss = 2624.98
INFO:tensorflow:global_step/sec: 68.4047
INFO:tensorflow:step = 301, loss = 2299.11
INFO:tensorflow:global_step/sec: 65.5281
INFO:tensorflow:step = 401, loss = 6921.98
INFO:tensorflow:global_step/sec: 67.7311
INFO:tensorflow:step = 501, loss = 1580.12
INFO:tensorflow:global_step/sec: 67.0255
INFO:tensorflow:step = 601, loss = 915.381
INFO:

DNNClassifier(params={'feature_columns': (_RealValuedColumn(column_name='', dimension=24, default_value=None, dtype=tf.float32, normalizer=None),), 'gradient_clip_norm': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x000001CB16775CC0>, 'optimizer': None, 'dropout': None, 'hidden_units': [30, 50, 20, 10], 'activation_fn': <function relu at 0x000001CB1894ED90>, 'input_layer_min_slice_size': None, 'embedding_lr_multipliers': None})

In [5]:
# Define the test inputs
def get_test_inputs():
    test_set.target[test_set.target < 5] = 0
    test_set.target[test_set.target > 5] = 1
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    return x, y

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=get_test_inputs, steps=1)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

# Classify two new flower samples.
def new_samples():
    return np.array(
  [[5.28923887058821E+01,3.04600220000000E+07,1.63233867085543E+00,4.84632109888550E+01,5.17815581804496E+00,1.92868014777594E+01,6.79631210347290E+01,6.74697763553599E+00,1.34159118941963E+02,3.20368789652710E+01,6.97565625034177E+00,6.94642591592048E+00,9.50779920000000E+07,4.02370560829584E+01,2.74199628024623E+00,4.64485044838654E+01,4.54869433081422E+01,4.74570928049574E+01,4.74301269937098E+01,4.96088497279859E+01,8.23161421257336E+01,8.92217139172395E+01,4.35389734769129E+01,6.46179700000000E+07],
   [5.20657577073686E+01,3.54777630000000E+07,1.57343623734931E+00,5.02344036142858E+01,5.26417871495310E+00,1.78885490585671E+01,6.56435137434607E+01,6.96815521546032E+00,1.34105211273476E+02,3.43564862565393E+01,7.57422160886158E+00,6.97889297590332E+00,1.03263656000000E+08,4.24020583563869E+01,2.81190562726906E+00,4.81650989725272E+01,4.78622276218487E+01,4.66117901366285E+01,4.91743524731534E+01,4.96060327349811E+01,8.50966572029520E+01,9.24149155513926E+01,4.43062195793519E+01,6.77858930000000E+07]], dtype=np.float32)

predictions = list(classifier.predict(input_fn=new_samples))

print(
  "New Samples, Class Predictions:    {}\n"
  .format(predictions))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-04-07:21:52
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-05-04-07:21:52
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.70241, accuracy/baseline_label_mean = 0.422088, accuracy/threshold_0.500000_mean = 0.70241, auc = 0.756859, global_step = 2000, labels/actual_label_mean = 0.422088, labels/prediction_mean = 0.571336, loss = 77.0289, precision/positive_threshold_0.500000_mean = 0.610714, recall/positive_threshold_0.500000_mean = 0.813511

Test Accuracy: 0.702410

New Samples, Class Predictions:    [0, 1]

